In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI RAG Engine with Vertex AI Feature Store

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_feature_store.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Frag-engine%2Frag_engine_feature_store.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/rag-engine/rag_engine_feature_store.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_feature_store.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Holt Skinner](https://github.com/holtskinner) |

## Overview

This notebook illustrates how to use [Vertex AI RAG Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview) with [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore/latest/overview) as a vector database.

RAG Engine uses a built-in vector database powered by Spanner to store and manage vector representations of text documents. The vector database retrieves relevant documents based on the documents' semantic similarity to a given query.

By integrating Vertex AI Feature Store as an additional vector database, RAG Engine can use Vertex AI Feature Store to handle large data volumes with low latency, which helps to improve the performance and scalability of your RAG applications.

For more information, refer to the [official documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/use-vertexai-vector-search).

For more details on RAG corpus/file management and detailed support please visit [Vertex AI RAG Engine API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api)

## Get started

### Install Vertex AI SDK and other required packages


In [ ]:
%pip install --force-reinstall --quiet git+https://github.com/googleapis/python-aiplatform.git@copybara_696165713
%pip install --upgrade --user --quiet google-cloud-bigquery

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import Libraries

In [ ]:
from google.cloud import bigquery
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
from vertexai.resources.preview import feature_store

## Set up Vertex AI Feature Store

Vertex AI Feature Store, a managed cloud-native service, is an essential component of Vertex AI. It simplifies machine learning (ML) feature management and online serving by letting you manage feature data within a BigQuery table or view. This enables low-latency online feature serving.

For `FeatureOnlineStore` instances created with optimized online serving, you
can take advantage of a vector similarity search to retrieve a list of
semantically similar or related entities, which are known as
*approximate nearest neighbors*.

The following sections show you how to set up a Vertex AI Feature Store instance for your RAG application.


### Create a BigQuery table schema

Use the Cloud Console or the code below to create a BigQuery table schema. It
must contain the following fields to serve as the data source.

| Field name | Data type | Status |
|-------------|-----------|--------|
| `corpus_id` | `String` | Required |
| `file_id` | `String` | Required |
| `chunk_id` | `String` | Required |
| `chunk_data_type` |`String` | Nullable |
| `chunk_data` | `String` | Nullable |
| `file_original_uri` | `String` | Nullable |
| `embeddings` | `Float` | Repeated |


In [ ]:
client = bigquery.Client(project=PROJECT_ID)

# Define dataset and table name
dataset_id = "input_us_central1"  # @param {type:"string"}
table_id = "rag_source_new"  # @param {type:"string"}

schema = [
    bigquery.SchemaField("corpus_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("file_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("chunk_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("chunk_data_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("chunk_data", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("file_original_uri", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("embeddings", "FLOAT64", mode="REPEATED"),
]

dataset_ref = bigquery.DatasetReference(PROJECT_ID, dataset_id)

try:
    dataset = client.get_dataset(dataset_ref)
    print(f"Dataset {dataset_id} already exists.")
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Set the location (optional, adjust if needed)
    dataset = client.create_dataset(dataset)
    print(f"Created dataset {dataset.dataset_id}")

table_ref = dataset_ref.table(table_id)
table = client.create_table(bigquery.Table(table_ref, schema=schema))
print(f"Created table {PROJECT_ID}.{dataset_id}.{table_id}")

In [ ]:
BIGQUERY_TABLE = f'bq://{table.full_table_id.replace(":", ".")}'

### Provision a `FeatureOnlineStore` instance

To enable online serving of features, use the `CreateFeatureOnlineStore` API to set up a `FeatureOnlineStore` instance. If you
are provisioning a `FeatureOnlineStore` for the first time, the operation might
take approximately five minutes to complete.

In [ ]:
FEATURE_ONLINE_STORE_ID = "your_feature_online_store_id"  # @param {type: "string"}

fos = feature_store.FeatureOnlineStore.create_optimized_store(FEATURE_ONLINE_STORE_ID)

### Create a `FeatureView` resource

To connect the BigQuery table, which stores the feature data source, to
the `FeatureOnlineStore` instance, call the `CreateFeatureView` API to create a
`FeatureView` resource. When you create a `FeatureView` resource, choose the
default distance metric `DOT_PRODUCT_DISTANCE`, which is defined as the
negative of the dot product (smaller `DOT_PRODUCT_DISTANCE` indicates higher
similarity).

In [ ]:
FEATURE_VIEW_ID = "your_feature_view_id"  # @param {type: "string"}
fv = fos.create_feature_view(
    name=FEATURE_VIEW_ID,
    source=feature_store.utils.FeatureViewVertexRagSource(uri=BIGQUERY_TABLE),
)

In [ ]:
# Check that Feature View was created
print(fv)

## Use Vertex AI Feature Store in RAG Engine

After the Feature Store instance is set up, the following
sections show you how to set it up as the vector database to use with the RAG
application.

### Set the vector database to create a RAG corpus

To create the RAG corpus, you must use `FEATURE_VIEW_RESOURCE_NAME`. The
RAG corpus is created and automatically associated with the
Vertex AI Feature Store instance.

RAG APIs use the generated `rag_corpus_id` to handle the data upload to the Vertex AI Feature Store
instance and to retrieve relevant contexts from the `rag_corpus_id`.

In [ ]:
vector_db = rag.VertexFeatureStore(resource_name=fv.resource_name)

# Name your corpus
DISPLAY_NAME = "Feature Store Corpus"  # @param  {type:"string"}

# Create RAG Corpus
rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

In [ ]:
rag_corpus

## Import files into the BigQuery table using the RAG API

Use the `ImportRagFiles` API to import files from Google Cloud Storage or
Google Drive into the BigQuery table of the Vertex AI Feature Store
instance. The files are embedded and stored in the BigQuery table.

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Google Cloud Storage bucket

In [ ]:
GCS_BUCKET = "cloud-samples-data/gen-app-builder/search/cymbal-bank-employee"  # @param {type:"string"}

response = rag.import_files(  # noqa: F704
    corpus_name=rag_corpus.name,
    paths=[GCS_BUCKET],
    chunk_size=512,
    chunk_overlap=50,
)

In [ ]:
# Check the files just imported. It may take a few seconds to process the imported files.
rag.list_files(corpus_name=rag_corpus.name)

### Run a synchronization process to construct a `FeatureOnlineStore` index {:#run-sync-process}

After uploading your data into the BigQuery table, run a
synchronization process to make your data available for online serving. You must
generate a `FeatureOnlineStore` index using the `FeatureView`, and the
synchronization process might take 20 minutes to complete.


In [ ]:
feature_view_sync = fv.sync()
feature_view_sync

In [ ]:
# Optional: Wait for sync to complete
feature_view_sync.wait()

## Use your RAG Corpus to add context to your Gemini queries

When retrieved contexts similarity distance < `vector_distance_threshold`, the contexts (from `RagStore`) will be used for content generation.

In [ ]:
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                )
            ],
            similarity_top_k=10,
            vector_distance_threshold=0.4,
        ),
    )
)

rag_model = GenerativeModel("gemini-1.5-flash", tools=[rag_retrieval_tool])

In [ ]:
GENERATE_CONTENT_PROMPT = "What is RAG and why it is helpful?"  # @param {type:"string"}

response = rag_model.generate_content(GENERATE_CONTENT_PROMPT)

In [ ]:
response.text

## Using other generation APIs with RAG Retrieval Tool

The retrieved contexts can be passed to any SDK or model generation API to generate final results.

In [ ]:
RETRIEVAL_QUERY = "What is RAG and why it is helpful?"  # @param {type:"string"}

response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
        )
    ],
    text=RETRIEVAL_QUERY,
    similarity_top_k=10,
)

# The retrieved context can be passed to any SDK or model generation API to generate final results.
retrieved_context = " ".join(
    [context.text for context in response.contexts.contexts]
).replace("\n", "")

retrieved_context

## Cleaning up

Clean up resources created in this notebook.

In [ ]:
delete_rag_corpus = False  # @param {type:"boolean"}

if delete_rag_corpus:
    rag.delete_corpus(name=rag_corpus.name)